### During pix2pix training, I've already filtered for whitespace > 70 tiles and have the non-whitespace tiles saved in respective HE and US xlsx's. Do the same, but only for US. Label the whitespace > 70 them as 1, or "white". Then, use the all_ns_path and get the rest of the file paths and then label them as 0 ("not white"). Save a total US excel with their respective image paths and their label.


In [2]:
from glob import glob
import os
from natsort import natsorted
from tqdm import tqdm
import math
import random
import pandas as pd
import numpy as np

In [2]:
nspath = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US'
all_ns_path = glob(os.path.join(nspath,'*','*.png')) # list of all images
all_ns_path = natsorted(all_ns_path)

['\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_787xy0004.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_1811xy0005.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_2835xy0006.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_3859xy0007.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_4883xy0008.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_5907xy0009.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_6931xy0010.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_7955xy0011.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_8979xy0012.png',
 '\\\\shelter\\Kyu\\unstain2stain\\tiles\\registered_tiles\\US\\OTS_14684_1\\8801_10003xy0013.png',
 '\\\\shel

In [16]:
non_white_xlsx = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE\composition_per_tile\ns_train_image_path.xlsx"
non_white_df = pd.read_excel(non_white_xlsx)
non_white_names = non_white_df.iloc[:,1]
non_white_names = list(non_white_names)
print(len(non_white_names))
num_zeros = np.zeros(len(non_white_names),dtype=int)
my_dict = {'imagepath': non_white_names, 'label': num_zeros}
non_white_df = pd.DataFrame(my_dict)
non_white_df

79271


,imagepath,label
0,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
1,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
2,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
3,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
4,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
...,...,...
79266,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
79267,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
79268,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
79269,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0


In [18]:
white_names = set(all_ns_path) - set(non_white_names)
print(len(white_names))
num_ones = np.ones(len(white_names),dtype=int)
white_names = list(white_names)
my_dict2 = {'imagepath': white_names, 'label': num_ones}
white_df = pd.DataFrame(my_dict2)

195376


,imagepath,label
0,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
1,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
2,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
3,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
4,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
...,...,...
195371,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195372,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195373,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195374,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1


In [19]:
# concat the two df's and save:
df_src = r'\\fatherserverdw\Kevin\unstained_blank_classifier'
us_df = pd.concat([non_white_df,white_df],axis=0)
us_df

,imagepath,label
0,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
1,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
2,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
3,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
4,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
...,...,...
195371,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195372,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195373,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
195374,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1


In [20]:
us_df.to_excel(os.path.join(df_src,'train_df.xlsx'))

### Now do the same, but create a test_df. This will be just handpicked from the WSI's that was not included in the train_df:

In [1]:
from glob import glob
import os
from natsort import natsorted
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
# Test image comes from 5 WSI's not included in train_df: OTS_14684_4, OTS_14832_3,4,6,7, but only find ones that are not weird/corrupted and properly matched.

In [3]:
# Custom find test images:
# First 416 images of OTS_14832_4 US is all blank (xy0001~xy0416)
test_df_src = r'\\fatherserverdw\Kevin\unstained_blank_classifier\test_df.xlsx' # this was just an empty df with column name imagepath and label
test_df = pd.read_excel(test_df_src)
test_df_blank = test_df.copy(deep=True)
blank_test_samples_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US\OTS_14684_4"
blank_test_samples = os.listdir(blank_test_samples_src)[0:416]
blank_test_samples_path = [os.path.join(blank_test_samples_src,x) for x in blank_test_samples]
test_df["imagepath"] = blank_test_samples_path
test_df["label"] = np.ones(416)
test_df["label"] = test_df["label"].apply(lambda x: int(x))

In [4]:
# OTS_14832_4 xy0494~499,502~504,571~585,648~669, 742~749, 802~828,878~907,955~972, 1031~1059, 1109~1129, 1132~1145, 1186~1224 #232 samples

In [5]:
non_blank_test_samples_src = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US\OTS_14684_4"
non_blank_test_samples = os.listdir(non_blank_test_samples_src)
non_blank_test_sample_list = non_blank_test_samples[493:499] + non_blank_test_samples[501:504]+ non_blank_test_samples[570:585] + non_blank_test_samples[647:669]+ non_blank_test_samples[741:749]+ non_blank_test_samples[801:828]+ non_blank_test_samples[877:907]+ non_blank_test_samples[954:972]+ non_blank_test_samples[1030:1059]+ non_blank_test_samples[1108:1129]+ non_blank_test_samples[1131:1145]+ non_blank_test_samples[1185:1224]
list1 = [os.path.join(non_blank_test_samples_src,x) for x in non_blank_test_sample_list]

In [6]:
# OTS_14832_3 xy1570~1591, 1648~1668, 1724~1744, 1802~1820, 1879~1901, 1954~1979
non_blank_test_samples_src1 = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US\OTS_14832_3"
non_blank_test_samples1 = os.listdir(non_blank_test_samples_src1)
non_blank_test_sample_list1 = non_blank_test_samples1[1567:1589] + non_blank_test_samples1[1645:1666] + non_blank_test_samples1[1721:1742] + non_blank_test_samples1[1799:1818] + non_blank_test_samples1[1876:1899] + non_blank_test_samples1[1951:1977]
list2 = [os.path.join(non_blank_test_samples_src1,x) for x in non_blank_test_sample_list1]
all_non_blank_test_sample_list = list1 + list2 # 364 length

In [7]:
test_df_blank["imagepath"] = all_non_blank_test_sample_list
test_df_blank["label"] = np.zeros(364)
test_df_blank["label"] = test_df_blank["label"].apply(lambda x: int(x))

In [8]:
final_test_df = pd.concat([test_df,test_df_blank],axis=0)
final_test_df

,imagepath,label
0,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
1,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
2,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
3,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
4,\\shelter\Kyu\unstain2stain\tiles\registered_t...,1
...,...,...
359,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
360,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
361,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0
362,\\shelter\Kyu\unstain2stain\tiles\registered_t...,0


In [9]:
final_test_df.to_excel(test_df_src)